In [3]:
import bnn
import os
import cv2
import json
from PIL import Image
from IPython.display import display
from pynq import Xlnk
import pynq
import numpy as np
import yaml 
import math
import time
import requests
import matplotlib.pyplot as plt


output_path = '/home/xilinx/jupyter_notebooks/bnn/segment/'
data_test_path = '/home/xilinx/jupyter_notebooks/bnn/data_test/'

#Instantiate the classifier:
# hw_classifier = bnn.CnvClassifier(bnn.NETWORK_CNVW1A1,'cifar10',bnn.RUNTIME_HW)
# hw_classifier = bnn.CnvClassifier(bnn.NETWORK_CNVW1A2,'cifar10',bnn.RUNTIME_HW)
hw_classifier = bnn.CnvClassifier(bnn.NETWORK_CNVW2A2,'cifar10',bnn.RUNTIME_HW)

npy_file = os.path.join(data_test_path, 'data_test_compile.npz')
npy = np.load(npy_file)
data = {'data' : [], 'label' : []}
data['data'] = npy['data']
data['label'] = npy['label']
print(len(data['data']))

counter = 0
period = 2500
confusion = [0, 0, 0, 0]
images = []

log = open ( './result_log/2501_cnvW2A2_compile.csv', 'w' )
for npz in data['data']:
    t = (32,32,3)   
    # Creation of Array
    arr = np.zeros(t, dtype=np.uint8)
    pos = 0
    # RED CHANNEL   
    for i in range(32):
        for j in range(32):
            arr[i][j][2] = npz[pos]
            pos += 1    
    # GREEN CHANNEL        
    for i in range(32):
        for j in range(32):
            arr[i][j][1] = npz[pos]
            pos += 1
    # BLUE CHANNEL
    for i in range(32):
        for j in range(32):
            arr[i][j][0] = npz[pos]
            pos += 1

    img = Image.fromarray(arr,"RGB")
    path = './segment/img.jpg' #+ str(counter) + '.jpg'
    img.save(path)
    img = cv2.imread(path)
    img = cv2.resize(img, (32,32))
    images.append(img)

    if (counter%period == 0 or counter == len(data['data'])-1):
#         period += period + 250
        print(counter)
        print(len(images))
        print(period)
        inference = -time.time()
        rankingW2A2 = hw_classifier.classify_images_details(images)
        inference += time.time()
#         print("\n{: >10}{: >13}".format("[CLASS]","[RANKING]"))
#         for i in range(len(images)):
#             for j in range (0, len(hw_classifier.classes)):
#                 print("{: >10}{: >10}".format(hw_classifier.classes[j],rankingW2A2[i*2+j]))
        for i in range (0, len(images)):
#             predict = rankingW2A2[i]
            actual = data['label'][i+counter-len(images)+1]
            predict = 0
            
            empty = rankingW2A2[i*len(hw_classifier.classes)]/100
            occupied = rankingW2A2[i*len(hw_classifier.classes)+1]/100
            xs = [empty, occupied]
#             print(xs)
            percent = np.exp(xs) / sum(np.exp(xs))
#             percent = math.exp(occupied) / (math.exp(occupied) + math.exp(empty))
            
            if (occupied > empty):
                predict = 1

            result = predict*2 + actual
            index = result[0]
#             print(actual[0], percent, empty, occupied)
#             print(np.exp(xs) / sum(np.exp(xs)))~
            log.write('%d\t%.10f\t%d\t%.10f\t%.2f\t%.2f\n' % (actual[0], percent[1], not bool(actual[0]), percent[0], empty, occupied))

#             if index == 1 or index == 2:
#                 path = './data_image/data_test_pklot/img' + str(actual) + '_'+ str(i+counter-len(images)+1) + '.jpg'
#                 img = cv2.resize(images[i], (64,64))
#                 print(i+counter-len(images)+1)
#                 cv2.imwrite(path, img)
            
            confusion[index] += 1
        
        print("FPS ", end=': ')
        print(len(images)/inference)
        print("True Negative  - False Negative - False Positive - True Positive", end=": ")
        print(confusion)
        print("Accuracy", end=': ')
        print((confusion[0]+confusion[3])/(counter+1))    
        print("Error", end=': ')
        print(1-(confusion[0]+confusion[3])/(counter+1))
        
        images = []
        
#     if counter == period :
#         break
    counter += 1

log.close()

print(counter)
print("True Negative  - False Negative - False Positive - True Positive", end=": ")
print(confusion)
print("Accuracy", end=': ')
print((confusion[0]+confusion[3])/(counter))
print("Error", end=': ')
print(1-(confusion[0]+confusion[3])/(counter))
  
xlnk = Xlnk()
xlnk.xlnk_reset()    




30000
0
1
2500
FPS : 70.39903321640175
True Negative  - False Negative - False Positive - True Positive: [1, 0, 0, 0]
Accuracy: 1.0
Error: 0.0
2500
2500
2500
FPS : 835.117935034047
True Negative  - False Negative - False Positive - True Positive: [2491, 0, 10, 0]
Accuracy: 0.9960015993602559
Error: 0.003998400639744104
5000
2500
2500
FPS : 841.6078754340467
True Negative  - False Negative - False Positive - True Positive: [4339, 1, 16, 645]
Accuracy: 0.9966006798640272
Error: 0.003399320135972772
7500
2500
2500
FPS : 839.1802927210448
True Negative  - False Negative - False Positive - True Positive: [5613, 1, 19, 1868]
Accuracy: 0.9973336888414878
Error: 0.0026663111585122357
10000
2500
2500
FPS : 839.9380036909736
True Negative  - False Negative - False Positive - True Positive: [6919, 2, 21, 3059]
Accuracy: 0.9977002299770023
Error: 0.002299770022997749
12500
2500
2500
FPS : 837.6275442141027
True Negative  - False Negative - False Positive - True Positive: [7229, 5, 21, 5246]
Accura

/usr/lib/python3/dist-packages/ipykernel_launcher.py:132: DeprecationWarning: pynq.Xlnk is deprecated and will be removed in 2.7 - use pynq.allocate instead
